In [1]:
!pip -q install transformers sentence_transformers
!pip -q install huggingface_hub langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 88.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.6/462.6 KB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 KB 4.4 MB/s eta 0:00:00


In [2]:
!pip install configparser

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import os
import configparser

data = configparser.ConfigParser()
data.read_file(open('/content/apidata.config'))

In [4]:
os.environ['HUGGINGFACEHUB_API_TOKEN'] = data["OPENAI"]["HFREAD"]

In [5]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

In [6]:
template = """ Question: {question}
Answer: Let's think step by step."""

prompt_sbs = PromptTemplate(template=template, input_variables=["question"])

In [7]:
%%time
llm_hugF_t5 = HuggingFaceHub(repo_id="google/flan-t5-xl",
                             model_kwargs={"temperature":0,
                                           "max_length":64})

llm_hugF_flan20 = HuggingFaceHub(repo_id="google/flan-ul2",
                             model_kwargs={"temperature":0.1,
                                           "max_length":256})

CPU times: user 122 ms, sys: 16.7 ms, total: 138 ms
Wall time: 704 ms


In [11]:
llm_gpt_neo = HuggingFaceHub(repo_id="togethercomputer/GPT-NeoXT-Chat-Base-20B",
                             model_kwargs={"temperature":0.01,
                                           "max_length":1000})

In [12]:
hug_t5_chain = LLMChain(prompt=prompt_sbs, 
                        llm=llm_hugF_t5)

hug_flan20_chain = LLMChain(prompt=prompt_sbs, 
                        llm=llm_hugF_flan20)

hug_gptNeo_chain = LLMChain(prompt=prompt_sbs, 
                        llm=llm_gpt_neo)

In [16]:
query1 = """There were 50 eggs in the basket. 
            15 hatched into chicks, five were taken away. 
            How much is remaining?"""

In [14]:
print(hug_flan20_chain.run(query))

There are 50 - 15 = 35 eggs left after 15 hatched. 35 - 5


In [ ]:
print(hug_flan20_chain.run(query))

In [18]:
def test_models(query):
  print("t5_reply",end='\n')
  print(hug_t5_chain.run(query))
  print("flan20_reply",end='\n')
  print(hug_flan20_chain.run(query))
  print("gptNeo_reply",end='\n')
  print(hug_gptNeo_chain.run(query))

In [22]:
test_models(query=query1)

t5_reply
There were 50 - 15 = 45 eggs that remained. 5 - 45 = 15 eggs were taken away. There were 45 - 15 = 30 eggs remaining. The final answer: 30.
flan20_reply
There are 50 - 15 = 35 eggs left after 15 hatched. 35 - 5
gptNeo_reply


KeyboardInterrupt: ignored

In [ ]:
query = "You are a "